In [1]:
import torch as torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        # ? super
        # ? Conv2d(?, ?, ?)
        # ? Linear(?, ?, ?)
        
        super(Net, self).__init__()
        # input channel 1, output channel 6, kernel size 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # ? x.view
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        # flat x
        x = x.view(-1, self.flat_feature(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
        
    def flat_feature(self, x):
        size = x.size()[1:]
        flat_feature = 1
        for i in size:
            flat_feature *= i
        return flat_feature

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(params[2].size())

torch.Size([16, 6, 5, 5])


In [8]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net.forward(input)
target = Variable(torch.arange(1, 11))
criter = nn.MSELoss()
loss = criter(out, target)

In [26]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions)
print(len(list(loss.grad_fn.next_functions)))
print(loss.grad_fn.next_functions[0][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)

((<AddmmBackward object at 0x1079d97f0>, 0),)
1
((<ExpandBackward object at 0x107951668>, 0), (<ThresholdBackward object at 0x1079515f8>, 0), (<TBackward object at 0x107951630>, 0))
((<AccumulateGrad object at 0x1079515f8>, 0),)
